In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('_info-data-discrete.txt', sep=' ', header=None, names=["nazwa systemu", "liczba atrybutow", "liczba obiektow"])
df

,nazwa systemu,liczba atrybutow,liczba obiektow
0,australian,15,690
1,car,7,1728
2,fertilityDiagnosis,10,100
3,german-credit,21,1000
4,heartdisease,14,270
5,hepatitis-filled,20,155
6,house-votes-84,17,435
7,mushroom-modified-filled,23,8124
8,nursery,9,12960
9,diabetes,9,768


In [3]:
systems_info = pd.read_csv('dane/car.txt', sep=' ')
systems_info

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc
...,...,...,...,...,...,...,...
1722,low,low,5more,more,med,med,good
1723,low,low,5more,more,med,high,vgood
1724,low,low,5more,more,big,low,unacc
1725,low,low,5more,more,big,med,good


In [4]:
attributes_type = pd.read_csv("dane/car-type.txt", sep=" ", header=None, names=["Attribute", "Type"])
attributes_type

,Attribute,Type
0,a1,s
1,a2,s
2,a3,s
3,a4,s
4,a5,s
5,a6,s


# 3) Wczytujemy wybrany system np. w C++ i zdobywamy następujące informacje o systemie (load selected decision system for instance in C++ and find the information):

## a) wypisujemy istniejące w systemie symbole klas decyzyjnych (find available decision classes),

In [5]:
decision_classes = systems_info.iloc[:, -1].unique()
print("Symbole klas decyzyjnych:", decision_classes)

Symbole klas decyzyjnych: ['unacc' 'acc' 'vgood' 'good']


## b) wielkości klas decyzyjnych (liczby obiektów w klasach) (find size of decision classes (number of objects in classes) ),

In [6]:
class_counts = systems_info.iloc[:, -1].value_counts()
print("Liczba obiektó w klasach:\n", class_counts)

Liczba obiektó w klasach:
 unacc    1209
acc       384
good       69
vgood      65
Name: unacc, dtype: int64


## c) minimalne i maksymalne wartości poszczególnych atrybutów(dotyczy atrybutów numerycznych) (minimal and maximal values for each attribute - apply for numerical attributes),

## d) dla każdego atrybutu wypisujemy liczbę różnych dostępnych wartości (for each attribute detect the number of different available values),

In [7]:
for column in systems_info.columns[:-1]:
    print(f"Columna '{column}' zawiera {systems_info[column].nunique()} unikalne wartości")


Columna 'vhigh' zawiera 4 unikalne wartości
Columna 'vhigh.1' zawiera 4 unikalne wartości
Columna '2' zawiera 4 unikalne wartości
Columna '2.1' zawiera 3 unikalne wartości
Columna 'small' zawiera 3 unikalne wartości
Columna 'low' zawiera 3 unikalne wartości


## e) dla każdego atrybutu wypisujemy listę wszystkich różnych dostępnych wartości(for each attribute list the set of different, available values),

In [8]:
for column in systems_info.columns[:-1]:
    print(f"Atrybut '{column}': {systems_info[column].unique()}")

Atrybut 'vhigh': ['vhigh' 'high' 'med' 'low']
Atrybut 'vhigh.1': ['vhigh' 'high' 'med' 'low']
Atrybut '2': ['2' '3' '4' '5more']
Atrybut '2.1': ['2' '4' 'more']
Atrybut 'small': ['small' 'med' 'big']
Atrybut 'low': ['med' 'high' 'low']


## f)odchylenie standardowe dla poszczególnych atrybutów w całym systemie i w klasach decyzyjnych (dotyczy atrybutów numerycznych) (compute standard deviation for each attribute in the whole system and separately for each decision class).

# 4) Wykonaj dla wybranych danych następujący preprocessing (do for selected data the following preprocessing):